# Careplan generation: Generating labels

**Author:** Eva Rombouts  
**Date:** 2024-07-21  
**Version:** 0.2

### Description
This script uses the new gpt-4o-mini model to generate care plans based on a set of care notes.

### Imports and constants

In [ ]:
# Environment setup
import os

def check_environment():
    try:
        import google.colab
        return "Google Colab"
    except ImportError:
        return "Local Environment"

env = check_environment()

if env == "Google Colab":
    print("Running in Google Colab")
    !pip install -q langchain langchain_core langchain_openai langchain_community
    from google.colab import drive, userdata
    drive.mount('/content/drive')
    os.chdir('/content/drive/My Drive/Colab Notebooks/GenCareAI/scripts')
    OPENAI_API_KEY = userdata.get('GCI_OPENAI_API_KEY')
else:
    print("Running in Local Environment")
    # !pip install -q
    from dotenv import load_dotenv
    load_dotenv()
    OPENAI_API_KEY = os.getenv('GCI_OPENAI_API_KEY')


In [ ]:
import os
import pandas as pd
import json
from tqdm import tqdm
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.callbacks import get_openai_callback

In [ ]:
# Constants and Configurations
MODEL = 'gpt-4o-mini-2024-07-18'
TEMP = 0.6

### Data

In [ ]:
df = pd.read_csv('../data/df_weeknotes.csv')

In [ ]:
class CarePlanItem(BaseModel):
    probleem: str = Field(description="Een enkel zorgprobleem.")
    doel: str = Field(description="Korte en bondige beschrijving van het doel")
    acties: List[str] = Field(description="De acties zoals beschreven in de rapportages")

class CarePlan(BaseModel):
    zorgplan: List[CarePlanItem]

In [ ]:
model= ChatOpenAI(api_key=OPENAI_API_KEY, temperature=TEMP, model=MODEL)

In [ ]:
pyd_parser = PydanticOutputParser(pydantic_object=CarePlan)

In [ ]:
test = df['notes'][1]

In [ ]:
PT = PromptTemplate(
    template = """
Hieronder staan rapportages van een verpleeghuisclient met dementie.
Schrijf een zorgplan voor de drie belangrijkste zorgproblemen op basis van de rapportages.
Schrijf het kort en zakelijk op.

RAPPORTAGES:
{notes}

{format_instructions}
""",
    input_variables=["notes"],
    partial_variables={"format_instructions": pyd_parser.get_format_instructions()},
)

# Format the prompt for the example library
P = PT.format(notes=test)
print(P)

In [ ]:
chain = PT | model | pyd_parser

In [ ]:
result = chain.invoke({"notes": test})

In [ ]:
# Print het resultaat in JSON-formaat
print(result.json(indent=2))

In [ ]:
with get_openai_callback() as cb:
    data = []
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):  # Gebruik tqdm om de progress bar te tonen
        result = chain.invoke({"notes": row['notes']})
        for item in result.zorgplan:
            data.append({
                'ct_id': row['ct_id'],
                'month': row['month'],
                'iteration': row['iteration'],
                'probleem': item.probleem,
                'doel': item.doel,
                'acties': '; '.join(item.acties)  # Acties als een samengevoegde string
            })

    # Opslaan van de verzamelde data in JSON formaat
    with open('../data/df_careplans_generated2.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print("Data is succesvol opgeslagen in JSON formaat.")
    print(cb)

In [ ]:
generated_data = pd.read_json('../data/df_careplans_generated2.json')

grouped = generated_data.groupby(['ct_id', 'month', 'iteration']).apply(lambda group: "\n\n".join(
    f"Probleem: {row['probleem']}\nDoel: {row['doel']}\nActies: {row['acties']}"
    for _, row in group.iterrows()
), include_groups=False).reset_index(name='careplan')

df_careplans = df.merge(grouped, on=['ct_id', 'month', 'iteration'], how='left')

In [ ]:
df_careplans.head()

In [ ]:
print(df_careplans.iloc[0,4])

In [ ]:
df_careplans.info()

In [ ]:
df_careplans.to_csv('../data/df_careplans.csv', index=False)